In [3]:
import pandas as pd

In [4]:
chatbotData = pd.read_csv("ChatData.csv", encoding='utf-8')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

In [5]:
len(answer)

11823

## text generation

In [7]:
from konlpy.corpus import kolaw #헌법 데이터
from konlpy.tag import Okt #한국어 형태소 분석기
from nltk.tokenize import sent_tokenize #문장 단위로 토큰화

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [8]:
c = kolaw.open("constitution.txt").read()
len(c) #18884

18884

In [9]:
len(sent_tokenize(c)) #357

357

In [13]:
#konlpy.org => API => Okt
okt = Okt()
temp = sent_tokenize(c)[0:5]
for i in range(10):
    for w,t in okt.pos(temp[i]):
        print("w", w)
        print("t", t)

w 대한민국
t Noun
w 헌법
t Noun
w 


t Foreign
w 유구
t Noun
w 한
t Josa
w 역사
t Noun
w 와
t Josa
w 전통
t Noun
w 에
t Josa
w 빛나는
t Verb
w 우리
t Noun
w 대
t Modifier
w 한
t Modifier
w 국민
t Noun
w 은
t Josa
w 3
t Number
w ·
t Punctuation
w 1
t Number
w 운동
t Noun
w 으로
t Josa
w 건립
t Noun
w 된
t Verb
w 대한민국
t Noun
w 임시정부
t Noun
w 의
t Josa
w 법
t Noun
w 통과
t Noun
w 불의
t Noun
w 에
t Josa
w 항거
t Noun
w 한
t Josa
w 4
t Number
w ·
t Punctuation
w 19
t Number
w 민주
t Noun
w 이념
t Noun
w 을
t Josa
w 계승
t Noun
w 하고
t Josa
w ,
t Punctuation
w 조국
t Noun
w 의
t Josa
w 민주
t Noun
w 개혁
t Noun
w 과
t Josa
w 평화
t Noun
w 적
t Suffix
w 통일
t Noun
w 의
t Josa
w 사명
t Noun
w 에
t Josa
w 입
t Noun
w 각하
t Noun
w 여
t Josa
w 정의
t Noun
w ·
t Punctuation
w 인도
t Noun
w 와
t Josa
w 동포
t Noun
w 애
t Noun
w 로써
t Noun
w 민족
t Noun
w 의
t Josa
w 단결
t Noun
w 을
t Josa
w 공고
t Noun
w 히
t Adverb
w 하고
t Verb
w ,
t Punctuation
w 모든
t Noun
w 사회
t Noun
w 적
t Suffix
w 폐습
t Noun
w 과
t Josa
w 불의
t Noun
w 를
t Josa
w 타파
t Noun
w 하며
t Verb
w ,
t Punctuation
w 자율
t Noun
w 

IndexError: list index out of range

In [15]:
okt = Okt()
okt.pos('오늘은 날씨가 따뜻합니다')  #konlpy.org 참고

[('오늘', 'Noun'),
 ('은', 'Josa'),
 ('날씨', 'Noun'),
 ('가', 'Josa'),
 ('따뜻합니다', 'Adjective')]

In [16]:
doc0 = [" ".join(["".join(w) for w,t in okt.pos(s) if t not in ['Number', 'Foreign'] and w not in ['제','조']]) for s in sent_tokenize(c)]

In [17]:
len(doc0) 

357

In [18]:
doc0[2]

'대한민국 의 주권 은 국민 에게 있고 , 모든 권력 은 국민 으로부터 나온다 .'

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(doc0)

In [20]:
len(tokenizer.index_word) #1164 :  중복 제외한 단어 개수
# tokenizer.word_index #단어:인덱스

1164

In [21]:
import numpy as np
np.shape(tokenizer.texts_to_sequences(doc0))
doc = [a for a in tokenizer.texts_to_sequences(doc0) if len(a) > 1]

In [22]:
# for a in tokenizer.texts_to_sequences(doc0):
#     if len(a)<=1:
#         print(len(a))
# #     print("="*50)

In [23]:
len(doc)

354

In [24]:
maxLen = max([len(x) for x in doc])
# tokenizer.word_counts #각 단어가 몇번 언급됐는지
vocabSize = len(tokenizer.word_index) +1 
print(maxLen, vocabSize) #188 1165

188 1165


In [25]:
#padding
def genData(doc, maxLen, vocabSize):
    for sent in doc : #예를들어, sent의 길이가 7
        inputs=[]
        targets=[]
        for i in range(1, len(sent)): #1~6
            inputs.append(sent[0:i]) #[[45],[45,439],...]
            targets.append(sent[i]) #[[439],[648],...]
        y = np_utils.to_categorical(targets, vocabSize) #[439] => [[0,0,0,...1,0,0,0,...0]]
        inputSeq = sequence.pad_sequences(inputs, maxlen=maxLen) #padding
        yield(inputSeq,y) #객체 리턴
        
#         print("inputs:", inputs)
#         print("-"*50)
#         print("targets:", targets)
#         print("="*50)
#     return 

In [26]:
for i, (x,y) in enumerate(genData(doc, maxLen, vocabSize)):
    print(i)
    print("x", x.shape, "\n", x)
    print("y", y.shape, "\n", y)

0
x (187, 188) 
 [[  0   0   0 ...   0   0 102]
 [  0   0   0 ...   0 102  28]
 [  0   0   0 ... 102  28 602]
 ...
 [  0   0   0 ... 647 155   2]
 [  0   0 102 ... 155   2  20]
 [  0 102  28 ...   2  20 180]]
y (187, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1
x (6, 188) 
 [[  0   0   0 ...   0   0  45]
 [  0   0   0 ...   0  45 439]
 [  0   0   0 ...  45 439 648]
 [  0   0   0 ... 439 648 102]
 [  0   0   0 ... 648 102   4]
 [  0   0   0 ... 102   4 649]]
y (6, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
2
x (12, 188) 
 [[  0   0   0 ...   0   0 102]
 [  0   0   0 ...   0 102   1]
 [  0   0   0 ... 102   1 440]
 ...
 [  0   0   0 ...  34 651   4]
 [  0   0   0 ... 651   4  22]
 [  0   0   0 ...   4  22 331]]
y (12, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0.

 [0. 0. 0. ... 0. 0. 0.]]
81
x (7, 188) 
 [[  0   0   0 ...   0   0  34]
 [  0   0   0 ...   0  34  22]
 [  0   0   0 ...  34  22   4]
 ...
 [  0   0   0 ...   4 131   1]
 [  0   0   0 ... 131   1  64]
 [  0   0   0 ...   1  64   8]]
y (7, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
82
x (17, 188) 
 [[  0   0   0 ...   0   0  17]
 [  0   0   0 ...   0  17  10]
 [  0   0   0 ...  17  10 131]
 ...
 [  0   0   0 ...   2 224   9]
 [  0   0   0 ... 224   9  24]
 [  0   0   0 ...   9  24  13]]
y (17, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
83
x (14, 188) 
 [[  0   0   0 ...   0   0 131]
 [  0   0   0 ...   0 131 297]
 [  0   0   0 ... 131 297   1]
 ...
 [  0   0   0 ...  93 796   9]
 [  0   0   0 ... 796   9  24]
 [  0   0   0 ...   9  24  13]]
y (14, 

y (56, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
153
x (22, 188) 
 [[  0   0   0 ...   0   0  19]
 [  0   0   0 ...   0  19  10]
 [  0   0   0 ...  19  10 540]
 ...
 [  0   0   0 ...   2 114 107]
 [  0   0   0 ... 114 107 287]
 [  0   0   0 ... 107 287   3]]
y (22, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
154
x (37, 188) 
 [[  0   0   0 ...   0   0  19]
 [  0   0   0 ...   0  19  10]
 [  0   0   0 ...  19  10 151]
 ...
 [  0   0   0 ... 223   3  95]
 [  0   0   0 ...   3  95  12]
 [  0   0   0 ...  95  12  14]]
y (37, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
155
x (14, 188) 
 [[  0   0   0 ...   0   0 913]
 [  0   0   0 ... 

x (13, 188) 
 [[  0   0   0 ...   0   0  28]
 [  0   0   0 ...   0  28 252]
 [  0   0   0 ...  28 252   7]
 ...
 [  0   0   0 ...   7   9  50]
 [  0   0   0 ...   9  50  44]
 [  0   0   0 ...  50  44 403]]
y (13, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
226
x (22, 188) 
 [[  0   0   0 ...   0   0 242]
 [  0   0   0 ...   0 242   7]
 [  0   0   0 ... 242   7 411]
 ...
 [  0   0   0 ... 249   2  25]
 [  0   0   0 ...   2  25  11]
 [  0   0   0 ...  25  11 313]]
y (22, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
227
x (15, 188) 
 [[  0   0   0 ...   0   0  15]
 [  0   0   0 ...   0  15   1]
 [  0   0   0 ...  15   1 569]
 ...
 [  0   0   0 ... 135  18 250]
 [  0   0   0 ...  18 250  16]
 [  0   0   0 ... 250  16  29]]
y (15, 1165) 
 [[0. 1. 0. ... 0.

 [0. 0. 0. ... 0. 0. 0.]]
296
x (17, 188) 
 [[  0   0   0 ...   0   0  86]
 [  0   0   0 ...   0  86 120]
 [  0   0   0 ...  86 120   1]
 ...
 [  0   0   0 ...  20 417 418]
 [  0   0   0 ... 417 418  12]
 [  0   0   0 ... 418  12  14]]
y (17, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
297
x (13, 188) 
 [[  0   0   0 ...   0   0  86]
 [  0   0   0 ...   0  86 120]
 [  0   0   0 ...  86 120   4]
 ...
 [  0   0   0 ... 122   2 583]
 [  0   0   0 ...   2 583  12]
 [  0   0   0 ... 583  12  14]]
y (13, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
298
x (17, 188) 
 [[  0   0   0 ...   0   0  86]
 [  0   0   0 ...   0  86 120]
 [  0   0   0 ...  86 120   4]
 ...
 [  0   0   0 ... 186 158 311]
 [  0   0   0 ... 158 311  60]
 [  0   0   0 ... 311  60  41]]
y 

In [27]:
xdata=[]
ydata=[]
for x,y in genData(doc, maxLen, vocabSize):
    xdata.append(x)
    ydata.append(y)
xdata = np.concatenate(xdata)
ydata = np.concatenate(ydata)

In [28]:
xdata.shape
ydata.shape
# 배열 합치기
# a = np.array([[1,2],[3,4]])
# b = np.array([[5,6],[7,8],[9,10]])
# np.concatenate((a,b), axis=0)
# np.concatenate((a,b.T), axis=1)
# np.concatenate((a,b), axis=None)

(6917, 1165)

In [29]:
#모델 작성
model = Sequential()
model.add(Embedding(vocabSize, 100, input_length=maxLen)) #1165, 100
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(vocabSize, activation='softmax'))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 188, 100)          116500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1165)              117665    
Total params: 314,565
Trainable params: 314,565
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [72]:
hist = model.fit(xdata, ydata, epochs=500, batch_size=800)

Epoch 1/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.7194 - accuracy: 0.0467
Epoch 2/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6986 - accuracy: 0.0473
Epoch 3/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6788 - accuracy: 0.0468
Epoch 4/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6486 - accuracy: 0.0467
Epoch 5/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6139 - accuracy: 0.0493
Epoch 6/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.5706 - accuracy: 0.0536
Epoch 7/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.5337 - accuracy: 0.0505
Epoch 8/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.4928 - accuracy: 0.0520
Epoch 9/500
6917/6917 [==============================] - 17s 2ms/step - loss: 5.4556 - accuracy: 0.0526
Epoch 10/500
6917/6917 [==============================] - 16s 2m

6917/6917 [==============================] - 17s 2ms/step - loss: 3.4974 - accuracy: 0.2955
Epoch 80/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.4734 - accuracy: 0.3009
Epoch 81/500
6917/6917 [==============================] - 17s 2ms/step - loss: 3.4648 - accuracy: 0.3007
Epoch 82/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.4381 - accuracy: 0.3071
Epoch 83/500
6917/6917 [==============================] - 18s 3ms/step - loss: 3.4177 - accuracy: 0.3081
Epoch 84/500
6917/6917 [==============================] - 18s 3ms/step - loss: 3.3867 - accuracy: 0.3140
Epoch 85/500
6917/6917 [==============================] - 17s 3ms/step - loss: 3.3701 - accuracy: 0.3143
Epoch 86/500
6917/6917 [==============================] - 18s 3ms/step - loss: 3.3711 - accuracy: 0.3169
Epoch 87/500
6917/6917 [==============================] - 19s 3ms/step - loss: 3.3404 - accuracy: 0.3199
Epoch 88/500
6917/6917 [==============================] - 18s 3ms/st

6917/6917 [==============================] - 23s 3ms/step - loss: 1.6594 - accuracy: 0.6146
Epoch 234/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6568 - accuracy: 0.6104
Epoch 235/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6431 - accuracy: 0.6180
Epoch 236/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6490 - accuracy: 0.6099
Epoch 237/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6253 - accuracy: 0.6196
Epoch 238/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6319 - accuracy: 0.6193
Epoch 239/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6198 - accuracy: 0.6230
Epoch 240/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6218 - accuracy: 0.6146
Epoch 241/500
6917/6917 [==============================] - 23s 3ms/step - loss: 1.6200 - accuracy: 0.6286
Epoch 242/500
6917/6917 [==============================] - 2

6917/6917 [==============================] - 24s 3ms/step - loss: 0.9974 - accuracy: 0.7531
Epoch 388/500
6917/6917 [==============================] - 24s 3ms/step - loss: 1.0091 - accuracy: 0.7531
Epoch 389/500
6917/6917 [==============================] - 24s 3ms/step - loss: 0.9851 - accuracy: 0.7602
Epoch 390/500
6917/6917 [==============================] - 24s 3ms/step - loss: 1.0083 - accuracy: 0.7482
Epoch 391/500
6917/6917 [==============================] - 24s 4ms/step - loss: 0.9973 - accuracy: 0.7508
Epoch 392/500
6917/6917 [==============================] - 24s 4ms/step - loss: 0.9892 - accuracy: 0.7545
Epoch 393/500
6917/6917 [==============================] - 25s 4ms/step - loss: 0.9977 - accuracy: 0.7558
Epoch 394/500
6917/6917 [==============================] - 24s 4ms/step - loss: 0.9926 - accuracy: 0.7493
Epoch 395/500
6917/6917 [==============================] - 24s 3ms/step - loss: 0.9869 - accuracy: 0.7555
Epoch 396/500
6917/6917 [==============================] - 2

In [32]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.show()
model.save("testGen.hdf5")

NameError: name 'hist' is not defined

In [33]:
from keras.models import load_model
model = load_model("testGen.hdf5")

In [50]:
wordList="대한민국 의 국민 이 되는 요건 은 법률 로 정한 다 .".split()
wordList

['대한민국', '의', '국민', '이', '되는', '요건', '은', '법률', '로', '정한', '다', '.']

In [51]:
reverseWordMap = dict(map(reversed,tokenizer.word_index.items()))
len(reverseWordMap) #1164

1164

In [52]:
x = sequence.pad_sequences([[tokenizer.word_index[w] for w in wordList[:2]]], maxlen=maxLen)
x

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [53]:
np.shape(model.predict(x))  #(1, 1165)
p = model.predict(x)[0]

In [54]:
np.argsort(p)

array([790, 418, 937, ...,  22, 440, 441], dtype=int64)

In [56]:
idx = np.flip(np.argsort(p),0) #내림차순
p[idx] #다음에 나올 수 있는 각 단어들의 확률

array([2.0857935e-01, 1.6706514e-01, 1.0696493e-01, ..., 8.2888029e-08,
       8.0847720e-08, 5.6715280e-08], dtype=float32)

In [45]:
x = np.array([2.5, 1.5, 0, 1])
x.argsort()  #[2, 3, 1, 0] : 인덱스 번호

array([2, 3, 1, 0], dtype=int64)

In [57]:
for i in idx[:10]:
    print(reverseWordMap[i])

영토
주권
국민
모든
영역
통일
자백
평화
경
법


In [61]:
def predictWord(i, n):
    x = sequence.pad_sequences([[tokenizer.word_index[w] for w in wordList[:i]]], maxlen=maxLen)
    p = model.predict(x)[0]
    idx = np.flip(np.argsort(p),0) #내림차순 정렬
    for j in idx[:n]:
        print(" ".join(wordList[:i]), reverseWordMap[j], "(p={:4.2f}%)".format(100*p[j]))

In [62]:
predictWord(1,3) #'대한민국' 다음 단어? => 3개 추천
print("="*50)
predictWord(2,3) #'대한민국 의' 다음 단어? => 3개 추천
print("="*50)
predictWord(5,3) #"대한민국의 국민이 되는" 다음 단어? => 3개 추천


대한민국 의 (p=59.51%)
대한민국 은 (p=31.45%)
대한민국 헌법 (p=2.42%)
대한민국 의 영토 (p=20.86%)
대한민국 의 주권 (p=16.71%)
대한민국 의 국민 (p=10.70%)
대한민국 의 국민 이 되는 요건 (p=96.60%)
대한민국 의 국민 이 되는 국가 (p=0.60%)
대한민국 의 국민 이 되는 은 (p=0.45%)
